In [22]:
import os
import numpy as np
import pandas as pd

def csv_loader(drive, type):
    if type == 'train':
        data = pd.read_csv(f"{drive}:\\python__\\Final\\sample.csv")
    else:
        data = pd.read_csv(f"{drive}:\\python__\\Final\\valid.csv")
    
    if drive == 'd':
        data.name = data.name.str.replace("E","D")
    
    del data['Unnamed: 0']
    print("csv파일 로드 완료")
    
    return data

In [23]:
data = csv_loader('e','valid')

csv파일 로드 완료


In [24]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch

class dataset():

    def data_storage(drive, data, start, end):
        drive = drive.upper()

        if len(data) > 100000:
            folder_name = 'train'
        else:
            folder_name = 'valid'

        for i,j in zip(list(data.name)[start:end], range(start,end)):
            img = Image.open(i)
            img = img.resize((50,50))
            y = np.array(img)  # 이미지 -> 넘파이

            scaled_y0 = y[:,:,0]
            # scaled_y1 = y[:,:,1]
            # scaled_y2 = y[:,:,2]

            scaled_y0[(scaled_y0 >= 220)] = 255
            # scaled_y1[(scaled_y1 >= 220)] = 255
            # scaled_y2[(scaled_y2 >= 220)] = 255

            scaled_y0[(scaled_y0 < 220)] = 0
            # scaled_y1[(scaled_y1 < 220)] = 0
            # scaled_y2[(scaled_y2 < 220)] = 0

            scaled_y0 = torch.from_numpy(scaled_y0)
            # scaled_y1 = torch.from_numpy(scaled_y1)
            # scaled_y2 = torch.from_numpy(scaled_y2)

            scaled_y0 = scaled_y0.numpy()

            img_scaled_y = Image.fromarray(scaled_y0) # NumPy array to PIL image
            
            img_scaled_y.save(f'./{folder_name}_data/{j}.png','png')
            
            if j % 3000 == 0:
                print(f"{j}개째 완료")
            
        print(f"{end}개 완료")

    
    def data_ready(drive, data):
        drive = drive.upper()
        
        if len(data) > 100000:
            folder_name = 'train'
        else:
            folder_name = 'valid'
        
        img_path_list = []
        for i in range(len(os.listdir(f"{drive}:\\python__\\Final\\{folder_name}_data\\"))):
            img_name = f"{i}.png"
            img_path = os.path.join(f"{drive}:\\python__\\Final\\{folder_name}_data\\", img_name)
            img_path_list.append(img_path)
        img_path_list

        # 데이터셋 텐서형태로 stack -> MNIST에서 tran.data와 같은 형태
        out_list = []
        for i in range(len(img_path_list)):
            img = Image.open(img_path_list[i])
#             img = img.resize((50,50))
            y = np.array(img)  # 이미지 -> 넘파이
            x = torch.from_numpy(y) # 넘파이 -> 텐서

            out_list.append(x)
            if i % 10000 == 0:
                print(f"{i}까지 진행됨.")
            
        train_data = torch.stack(out_list, dim = 0)

        print("데이터 구축 완료")
        print(train_data.size())
        
        return train_data
        
    
    def save_data_npy(data, label):
        if len(data) > 100000:
            folder_name = 'train'
        else:
            folder_name = 'valid'
            
        # 텐서를 외부파일로저장
        a = data
        a_1 = label
        
        a_np = a.numpy()
        a_np_1 = a_1.numpy()
        
        np.save(f'{folder_name}_data', a_np)
        np.save(f'{folder_name}_target', a_np_1)
        
        np_load = np.load(f'{folder_name}_data.npy')
        np_load_1 = np.load(f'{folder_name}_target.npy')
        
        result = torch.from_numpy(np_load)
        result_1 = torch.from_numpy(np_load_1)
        
        return result, result_1
    
    
    def load_data_npy(data, label):
        # 외부파일을 텐서로 불러오기
        np_load = np.load(data)
        np_load_1 = np.load(label)
        
        result = torch.from_numpy(np_load)
        result_1 = torch.from_numpy(np_load_1)
        
        return result, result_1

    
    def label_loader(data):
        if len(data) > 100000:
            type_data = train_data
            type_target = train_target
        else:
            type_data = valid_data
            type_target = valid_target
            
        target = data.label_index[:len(type_data)]
        type_target = torch.from_numpy(target.values)
        
        print("label 구축 완료")
        print(type_target.size())
        
        return type_target
        
        
    def plot(type, index):
        if type == 'valid':
            type_data = valid_data
        else:
            type_data = train_data
            
        x = type_data[index]
        img = (np.array(x.detach().cpu(), dtype='float'))

        plt.imshow(img, cmap='gray')
        plt.show()

In [25]:
train_data,train_target = dataset.load_data_npy('train_data.npy', 'train_target.npy')

In [26]:
valid_data,valid_target = dataset.load_data_npy('valid_data.npy', 'valid_target.npy')

# 함수 구현

In [27]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch

def check_mapping(index, type):
    if type == 'valid':
        type_data = valid_data
        type_target = valid_target
    else:
        type_data = train_data
        type_target = train_target
    
    print("index : "+ str(index))
    print("")
    print("##  csv 파일  ##")
    print(data.name[index])
    print(data.label[index])
    print()
    print("##  npy 파일  ##")
    print(type_data[index])
    print(type_target[index])
    
    print(data[data.label_index==(type_target[index].item())].label.iloc[0])
    

In [32]:
check_mapping(31532,'valid')

index : 31532

##  csv 파일  ##
E:\다양한 형태의 한글 문자 OCR\Validation\[원천]validation_필기체\1.글자\130\15230006065.jpg
뢨

##  npy 파일  ##
tensor([[255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255],
        ...,
        [255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255]], dtype=torch.uint8)
tensor(715)
뢨
